# Tumor-Only Workflow Benchmark
This project uses existing T/N calls to bench our beta Tumor-only workflow from:
https://github.com/kids-first/kf-tumor-workflow

In [1]:
import sevenbridges as sbg
import re
import sys
import requests
import pdb
from sevenbridges.http.error_handlers import rate_limit_sleeper, maintenance_sleeper
config = sbg.Config(profile='turbo')
api = sbg.Api(config=config, error_handlers=[rate_limit_sleeper, maintenance_sleeper])

## Copy Files
Using a manifest provided by Alvin. The PBTA portion has to be copied from Datasets while non-PBTA is copied from existing projects

### Copy PBTA
All files are in a subdir - list from there, then cross-reference with a namifest by file name

In [6]:
cp_project = 'd3b-bixu/kf-tumor-only-wf-dev'
pbta_manifest = open('/Users/brownm28/Documents/2022-Jul-5_T-O_benchmark/PBTA-only_manifest.tsv')
file_dict = {}
head = next(pbta_manifest)
header = head.rstrip('\n').split('\t')
n_idx = header.index('file_name')
e_idx = header.index('experiment_strategy')
for line in pbta_manifest:
    info = line.rstrip('\n').split('\t')
    (fname, exp) = (info[n_idx], info[e_idx])
    file_dict[fname] = exp


In [15]:
pbta_dataset_dir = api.files.get('61bcb355eb5e796b5a83ae28')
flist = pbta_dataset_dir.list_files().all()

In [16]:
pbta_list = []
for obj in flist:
    if obj.name in file_dict:
        pbta_list.append(obj)


In [18]:
for fobj in pbta_list:
    new_cp = fobj.copy(project=cp_project, name=fobj.name)
    new_cp.tags = ['BENCHMARK', file_dict[new_cp.name]]
    new_cp.save()

### Copy non-PBTA
File IDs from file_id column should be good enough

In [4]:
cp_project = 'd3b-bixu/kf-tumor-only-wf-dev'
non_pbta_manifest = open('/Users/brownm28/Documents/2022-Jul-5_T-O_benchmark/non-PBTA_filtered.txt')
head = next(non_pbta_manifest)
header = head.rstrip('\n').split('\t')
f_idx = header.index('file_id')
e_idx = header.index('experiment_strategy')
for line in non_pbta_manifest:
    info = line.rstrip('\n').split('\t')
    try:
        f_obj = api.files.get(info[f_idx])
        new_cp = f_obj.copy(project=cp_project, name=f_obj.name)
        new_cp.tags = ['BENCHMARK', info[e_idx]]
        new_cp.save()
    except Exception as e:
        print (e)
        pdb.set_trace()
        hold=1
